<a href="https://colab.research.google.com/github/bjarkedc/RAG_FLAN_T5/blob/main/t5_flan_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install rank-bm25 nltk transformers sentencepiece

In [ ]:
import pandas as pd
import re
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
from transformers import T5ForConditionalGeneration, T5Tokenizer
import string

nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
def parse_docs(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        file_content = file.read()

    # Regular expression to match <doc> elements
    doc_pattern = re.compile(r'<doc id="([^"]+)" url="([^"]+)" title="([^"]+)">(.*?)</doc>', re.DOTALL)

    # Find all matches
    matches = doc_pattern.findall(file_content)

    # Extract data and create a list of dictionaries
    docs = [{'id': match[0], 'url': match[1], 'title': match[2], 'text': match[3].strip()} for match in matches]
    return docs

# Create a DataFrame
def create_dataframe(docs):
    return pd.DataFrame(docs)

def is_only_special_chars(word):
    special_chars = set(string.punctuation)
    # Check if each character in the word is a special character
    return all(char in special_chars for char in word)

# Preprocess text: tokenize and remove stopwords
def preprocess(text):
    # Define a set of special characters
    special_chars = set(string.punctuation)

    # Initialize the stemmer
    stemmer = PorterStemmer()

    # Load set of English stopwords
    stop_words = set(stopwords.words('english'))

    # Tokenize and convert to lowercase
    tokens = word_tokenize(text.lower())

    # Preprocess tokens
    processed_tokens = []
    for token in tokens:
        if token not in stop_words and not all(char in special_chars for char in token):
            stemmed_token = stemmer.stem(token)
            processed_tokens.append(stemmed_token)

    return processed_tokens

file_path = 'drive/MyDrive/apps/data-anlp/starwarsfandomcom-removed-space-and.txt'  # Replace with the actual path to your file
docs = parse_docs(file_path)
df = create_dataframe(docs)
df['tokenized_text'] = df['text'].apply(preprocess)
df = df[df['title'] != df['text']]

In [29]:
model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-base')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
## dev set

dev_set_path = '/content/drive/MyDrive/data-anlp/star_wars_dataset_dev/'

# Function to read a file and return a list of lines
def read_file_to_list(file_path):
    with open(file_path, 'r') as file:
        return [line.strip() for line in file]

# Load the data
dev_questions = read_file_to_list(dev_set_path + 'questions.txt')
dev_answers = read_file_to_list(dev_set_path + 'answers.txt')

In [30]:
def robs_eval(gold, pred):
    """
    An answer is considered correct if at least half of the gold
    tokens are in the prediction. Note that this is a shortcut,
    and will favor long answers.
    """
    gold = set(gold.strip().lower().replace('.', '').split(' '))
    pred = set(pred.strip().lower().replace('.', '').split(' '))
    return len(gold.intersection(pred)) >= len(gold)/2

In [77]:
def query_bm25(query, num_results=10):
    query_tokens = preprocess(query)
    print(query_tokens)
    doc_scores = bm25.get_scores(query_tokens)
    top_doc_indices = sorted(range(len(doc_scores)), key=lambda i: doc_scores[i], reverse=True)[:num_results]
    return df.iloc[top_doc_indices]['text'].str.cat(sep=' ')

def generate_response(query, context):
    # Concatenate the query and context
    input_text = f'Answer this question: "{query}". Based off the following context: "{context}".'

    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Truncate the input to the model's max length if necessary
    max_length = tokenizer.model_max_length
    if input_ids.size(1) > max_length:
        input_ids = input_ids[:, :max_length]

    truncated_input_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    output_ids = model.generate(input_ids)[0]
    response = tokenizer.decode(output_ids, skip_special_tokens=True)
    return response, context, truncated_input_text

In [80]:
### grid search

from sklearn.model_selection import ParameterGrid
import numpy as np
from tqdm import tqdm

max_pairs = len(dev_questions)
#max_pairs = 10

param_grid = {
    'k1': np.linspace(0.75, 1.75, num=3),
    'b': np.linspace(0.3, 0.7, num=3)
}
grid = ParameterGrid(param_grid)
best_score = -1
best_params = None
results = {}  # Dictionary to store results

for params in tqdm(grid, desc="Grid Search"):
    print(f"\n{params}")
    bm25 = BM25Okapi(corpus=df['tokenized_text'].tolist(), k1=params['k1'], b=params['b'])
    total_correct = 0

    for query, answer in tqdm(zip(dev_questions[:max_pairs], dev_answers[:max_pairs]), total=len(dev_questions[:max_pairs]), desc="Evaluating Queries"):
        context = query_bm25(query, 15)
        response, context, truncated_input_text = generate_response(query, context)
        print(f"""
{query}
{response}
{answer}
----------------------------------------------""")

        if robs_eval(answer, response):
            total_correct += 1
            print("total = " + str(total_correct))

    # Save the results
    results[str(params)] = total_correct

    # Assess the effectiveness of the parameters
    print(f"Total Correct: {total_correct}")
    if total_correct > best_score:
        best_score = total_correct
        best_params = params

print(f"Best Params: {best_params}, Best Score: {best_score}")
# Print the results dictionary
print("\nAll Results:")
for param, score in results.items():
    print(f"{param}: {score}")


Grid Search:   0%|          | 0/9 [00:00<?, ?it/s]


{'b': 0.3, 'k1': 0.75}



Evaluating Queries:   0%|          | 0/154 [00:00<?, ?it/s]

['true', 'fals', 'r2-d2', 'origin', 'dialogu', 'actual', 'speak', 'part']



Evaluating Queries:   1%|          | 1/154 [00:03<09:05,  3.56s/it]

...


What is Finn's stormtrooper number?
two
FN-2187
----------------------------------------------
['rey', "'s", 'kylo', "'s", 'mind', 'connect']



Evaluating Queries:  99%|█████████▉| 153/154 [09:35<00:04,  4.13s/it]


Why are Rey's and Kylo's minds connected?
Wayfinder Wayfinders were ancient pieces of technology created by both the Jedi and Sith as
Supreme Leader Snoke connected them.
----------------------------------------------
['rey', 'relat', 'palpatin']



Grid Search: 100%|██████████| 9/9 [1:31:14<00:00, 608.32s/it]


How is Rey related to Palpatine?
She is Palpatine's sister
She's his granddaughter.
----------------------------------------------
Total Correct: 29
Best Params: {'b': 0.7, 'k1': 0.75}, Best Score: 31

All Results:
{'b': 0.3, 'k1': 0.75}: 24
{'b': 0.3, 'k1': 1.25}: 24
{'b': 0.3, 'k1': 1.75}: 29
{'b': 0.5, 'k1': 0.75}: 28
{'b': 0.5, 'k1': 1.25}: 27
{'b': 0.5, 'k1': 1.75}: 24
{'b': 0.7, 'k1': 0.75}: 31
{'b': 0.7, 'k1': 1.25}: 29
{'b': 0.7, 'k1': 1.75}: 29


In [82]:
test_set_path = '/content/drive/MyDrive/data-anlp/star_wars_dataset_test/'
test_questions = read_file_to_list(test_set_path + 'questions.txt')
test_answers = read_file_to_list(test_set_path + 'answers.txt')

bm25 = BM25Okapi(corpus=df['tokenized_text'].tolist(), k1=best_params['k1'], b=best_params['b'])

data = []

total_correct = 0

for query, answer in tqdm(zip(test_questions, test_answers), total=len(test_answers), desc="Processing"):
    context = query_bm25(query, 15)

    response, context, truncated_input_text = generate_response(query, context)

    # Print the text (optional, can be removed if not needed)
    print(f"""
{truncated_input_text}
{response}
{answer}
----------------------------------------------""")

    # Check if the response is correct
    if robs_eval(answer, response):
        total_correct += 1
        print("total = " + str(total_correct))

    data.append({'Query': query, 'Correct Answer': answer, 'Model Response': response})

df_results = pd.DataFrame(data)
print(f"Total correct answers: {total_correct}/{len(test_answers)}")

Processing:   0%|          | 0/233 [00:00<?, ?it/s]

["'may", 'fourth', 'offici', 'taglin', 'galaxy-wid', 'holiday', 'celebr', 'first', 'week', 'may', 'honor', 'sci-fi', 'franchis']


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Processing:   0%|          | 1/233 [00:05<19:29,  5.04s/it]


Answer this question: "'May the Fourth be with you' is the official tagline of the galaxy-wide holiday celebrated the first week of May in honor of what sci-fi franchise?". Based off the following context: "Empire Day This page is about the Imperial holiday. You may be looking for the "Star Wars Rebels" episode of the same name. Empire Day was the day that Sheev Palpatine declared himself Emperor of the First Galactic Empire, bringing an end to the Republic. It was also the day Ezra Bridger was born, and two days before the births of Luke and Leia Skywalker on Polis Massa.[] From the following year on, Empire Day became a holiday marking the anniversary of the rise of the Empire. Festivities of the holiday included a parade on various Imperial-controlled planets, the playing of the Imperial anthem, "Glory of the Empire", as well as the airing of Palpatine with two Royal Guards at the Imperial Senate, but the footage used made the Emperor seem younger and more physically appealing than

Processing:   1%|          | 2/233 [00:09<18:07,  4.71s/it]

...


Answer this question: "What 'Pulp Fiction' actor was George Lucas' second choice to play Han Solo and would have taken the role had Harrison Ford declined it?". Based off the following context: "Kit Hillier Kit Hillier is a British actor who appeared as a Rebel soldier Echo Base extra in. Hillier was originally slated to play Han Solo's tauntaun handler, but actor Harrison Ford, who played Solo, sadly removed Hillier from the role during filming rehearsal after Hillier proved inept at holding onto the mechanical tauntaun's reins. Hillier was then replaced by actor Richard Bonehill. Harrison Ford Harrison Ford (born July 13, 1942) is an -nominated actor and director most famous for his portrayals of the character Han Solo in the "Star Wars" saga as well as the Indiana Jones in the. Ford has also been the star of many high-grossing hit blockbusters such as and, which have distanced him from his famous "Star Wars" and "Indiana Jones" roles. At one point Ford had roles in the top five box

Processing: 100%|█████████▉| 232/233 [19:30<00:05,  5.00s/it]


Answer this question: "A British prince and the First Son of the U.S. fall in love despite enthusiastically disagreeing on which "Star Wars" movie is the best, in what bestselling 2019 novel by Casey McQuiston?". Based off the following context: "Avenger of Jakku Avenger of Jakku was a pseudonym used by a collector of Imperial artifacts following the fall of the Galactic Empire. By 28 ABY, fellow Imperial artifact enthusiast [[Senator]] [[Ransolm Casterfo]] had encountered Avenger of [[Jakku]], but considered the collector's moniker ridiculous despite suspecting that it was probably only role-play of a part the collector might play in a reenactment. Behind the scenes. Avenger of Jakku was first mentioned in the novel [[Bloodline (novel)|"Bloodline"]], which was written by [[Claudia Gray]] and released in [[2016]]. Notes and references. [[Category:Collectors]] [[Category:Individuals of unspecified gender]] [[Category:Individuals of unspecified species]] Peter Suschitzky Peter Suschitzk

Processing: 100%|██████████| 233/233 [19:34<00:00,  5.04s/it]


Answer this question: "The language spoken by the Ewoks is a combination of two real-life languages. With one guess, name either of these languages.". Based off the following context: "Gwig Gwig was a young male Ewok who lived on the forest moon of Endor. He came of age and received his hood—together with Malani and another wokling—at the Festival of Hoods. He was also amongst a group of young Ewoks and woklings to whom Chirpa told the story of Logray, Morag and the Sunstar. Behind the scenes. Gwig was a background character with no spoken lines in the "Ewoks" television series. His name appears only on two of Nelvana's animation model cels, one of which shows him wearing a hood and the other a bandana. Both versions of Gwig are present in the crowd of young Ewoks in "Sunstar vs. Shadowstone." He was one of six 8-inch plush woklings made by Kenner. Necropolitan language The Necropolitan language was a language spoken by the Humans inhabitants of Necropolis, a planet that served as a b

In [191]:
df

,Question,Correct Answer,Fine-Tuned Model Answer,Base-With-RAG Model Answer,Fine-Tuned-With-RAG Model Answer
0,'May the Fourth be with you' is the official t...,Star Wars,galaxy wars**,Star Wars,Empire
1,What 2018 Star Wars film focuses on the early ...,Solo,Chewbacca,Chewbacca,NaN
2,"""Rogue One"" was a 2016 standalone feature film...",Star Wars,Star Wars*,Star Wars,Star Wars
3,"What Admiral in the rebellion, a member of the...",Admiral Ackbar,"Admiral in the rebellion, a member of the Mon ...",Admiral John Marshall,Dac was a planet in the Calamari sector of the...
4,"Sure to mess up Star Wars Day, 63.4% of Texas ...",May,february,March,Serroco
...,...,...,...,...,...
228,What is Jedi Master Yoda's last name in the St...,He doesn't have one,walker,Del Gormo,NaN
229,"Grand Moff Tarkin, the commander of the Death ...",Peter Cushing,who played Tarkin in the film?,Peter Cushing,Peter Cushing
230,What 'Pulp Fiction' actor was George Lucas' se...,Christopher Walken,charlie harris,Richard Bonehill,Harrison Ford
231,A British prince and the First Son of the U.S....,"Red, White, & Royal Blue",The Princess and the Wookiee,Bloodline,Mortal Love
